# Experiment 3
Code Overview :

+ select three features manually. description - title - price
+ Bert to get embedding of non numeric features
+ PCA for dimention reduction
+ Without upsampling and augmentation to solve unbalanced dataset problem


In [1]:
!pip install hazm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.7/316.7 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 kB 26.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 29.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for nltk: filename=nltk-3.3-py3-none-any.whl size=1394490 sha256=8f790e4a229808debae22d16c9834caf863bc9a3c3f1c25f343f8ed79faaf6cf
  Stored in directory: /root/.cache/pip/wheels/ac/62/f6/88933dadcd64a1614894614aa68cf57c4b8e5256acb650b1f1
  Created wheel for libwapiti: filename=libwapiti-0.2.1-cp39-cp39-linux_x86_64.whl size=180369 sha256=078ae6b16d03dee31f180268178d6d8609b26d55d6ffae574fbf4a2abea7daf8
  Stored in directory: /root/.cache/pip/wheels/8e/ff/82/9326b96f96f47472e02c453697b225813e4650c0ed4df2cd49
Successfully built nltk libwapiti
  Attempting unin

In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 64.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 100.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 29.2 MB/s eta 0:00:00


In [3]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from hazm import Normalizer, sent_tokenize,word_tokenize,Lemmatizer,Stemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import datasets
import pandas as pd 
from sklearn.preprocessing import StandardScaler 
from sklearn.decomposition import PCA 
from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score , log_loss
import codecs
import torch
from transformers import BertTokenizer, BertModel, BertForMaskedLM

In [4]:
divar_df = pd.read_csv('divar_sample_10k.csv')

In [5]:
input_x = divar_df[np.setdiff1d(divar_df.columns, ['cat1'])]
output_y = divar_df['cat1']

## Preprocess non numeric features
+ Normalizing 
+ Remove Stopwords

In [6]:
normalizer = Normalizer()
stopwords = [normalizer.normalize(x.strip()) for x in codecs.open(
    'stopwords.txt', 'r', 'utf-8').readlines()]
stopwords.extend(['تماس', 'تلگرام', 'واتساپ', 'با سلام', 'سلام', 'فروش', 'فوری',
                 'قیمت', 'مقطوع', 'تخفیف', 'نو', 'در حد نو', 'خرید', 'اقساطی', 'تحویل', 'اقساط'])

In [7]:
for idx , sent in enumerate(input_x['desc']):
    sent = normalizer.normalize(sent)
    sent = sent.replace('$NUM' , ' ')
    input_x['desc'][idx] = '  '.join([str(elem) for elem in [t for t in word_tokenize(sent) if t not in stopwords]])
    if idx % 1000 == 0:
      print(idx)

<ipython-input-7-091faeded13c>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_x['desc'][idx] = '  '.join([str(elem) for elem in [t for t in word_tokenize(sent) if t not in stopwords]])


0
1000
2000
3000
4000
5000
6000
7000
8000
9000


In [8]:
for idx , sent in enumerate(input_x['title']):
    sent = normalizer.normalize(sent)
    input_x['title'][idx] = '  '.join([str(elem) for elem in [t for t in word_tokenize(sent) if t not in stopwords]])
    if idx % 1000 == 0:
      print(idx)

<ipython-input-8-3e008e98ddc5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_x['title'][idx] = '  '.join([str(elem) for elem in [t for t in word_tokenize(sent) if t not in stopwords]])


0
1000
2000
3000
4000
5000
6000
7000
8000
9000


## Bert Model

In [9]:
tokenizer = BertTokenizer.from_pretrained('HooshvareLab/bert-fa-zwnj-base')

model = BertModel.from_pretrained('HooshvareLab/bert-fa-zwnj-base' , output_hidden_states=True)
device = torch.device("cuda:0")                                                   
model = model.to(device) 
model.eval()
normalizer = Normalizer()

Some weights of the model checkpoint at HooshvareLab/bert-fa-zwnj-base were not used when initializing BertModel: ['cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [10]:
def get_embedding(text):
    # Tokenize our sentence with the BERT tokenizer.
    normalizedText = normalizer.normalize(text)
    tokenized_text = tokenizer.tokenize(normalizedText)

    segmentText = ["[CLS]"]
    segmentText += tokenized_text
    segmentText.append("[SEP]")

    segments_ids = [1] * len(segmentText)

    # Map the token strings to their vocabulary indeces.
    indexed_tokens = tokenizer.convert_tokens_to_ids(segmentText)

    tokens_tensor = torch.tensor([indexed_tokens]).to(device)
    segments_tensors = torch.tensor([segments_ids]).to(device)
    
    with torch.no_grad():
        output = model(tokens_tensor, segments_tensors)

    return output

In [11]:
desc_embeddings = []
for idx , sent in enumerate(input_x['desc']):
    desc_embeddings.append(get_embedding(sent)[1][0])
    if idx % 1000 == 0:
      print(idx)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000


In [12]:
title_embeddings = []
for idx , sent in enumerate(input_x['title']):
    title_embeddings.append(get_embedding(sent)[1][0])
    if idx % 1000 == 0:
      print(idx)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000


In [13]:
for idx , sent in enumerate(desc_embeddings):
  desc_embeddings[idx] = desc_embeddings[idx].cpu().numpy()

In [14]:
for idx , sent in enumerate(title_embeddings):
  title_embeddings[idx] = title_embeddings[idx].cpu().numpy()

In [15]:
desc_embeddings_df = pd.DataFrame(desc_embeddings)
title_embeddings_df = pd.DataFrame(title_embeddings)

## PCA for dimention reduction

In [16]:
pca = PCA(n_components = 100)
pca.fit(desc_embeddings_df)
desc_embeddings_df_pca = pca.transform(desc_embeddings_df)
desc_embeddings_df_pca = pd.DataFrame(desc_embeddings_df_pca,columns=[str(i) for i in range(0,100)])

In [17]:
pca = PCA(n_components = 100)
pca.fit(title_embeddings_df)
title_embeddings_df_pca = pca.transform(title_embeddings_df)
title_embeddings_df_pca = pd.DataFrame(title_embeddings_df_pca,columns=[str(i) for i in range(100,200)])

In [28]:
desc_embeddings_df.to_csv('desc_embeddings_df_pca.csv')
title_embeddings_df.to_csv('title_embeddings_df_pca.csv')

In [18]:
scalar = StandardScaler()
numeric_scaled_features = pd.DataFrame(scalar.fit_transform(input_x[['price']]) , columns=['price'])

In [19]:
input_x = pd.concat([desc_embeddings_df_pca , title_embeddings_df_pca , numeric_scaled_features], axis=1) 

In [20]:
all_data_df = pd.concat([input_x ,output_y], axis=1) 

all_data_df.to_csv('bert_pca_df.csv')

## Logistic Regression for classification

In [21]:
input_x.columns = input_x.columns.astype(str)
X_train, X_test, y_train, y_test = train_test_split(input_x, output_y, test_size=0.25, random_state=16)

In [22]:
logreg = LogisticRegression(random_state=16)
logreg.fit(X_train, y_train)

y_pred = logreg.predict(X_test)

/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [23]:
accuracy_score(y_test, y_pred)

0.8076